In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
import joblib

In [14]:
# 学習データの読み込み
train_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/train.csv")

# テストデータの読み込み
test_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/test.csv")

# 学習データとテストデータを連結する
df=pd.concat([train_df,test_df],ignore_index=True)
#nullのCSV化
df_null=df.isnull().sum()
df_null.to_csv("null.csv")

# 元データをコピー
df2=df.copy()

In [3]:
#null_csvを見ながら,nullのデータの確認
#sns.countplot("Survived",data=df2)
#sns.countplot("Age",data=df2)
#sns.countplot("Fare",data=df2)
#sns.countplot("Cabin",data=df2)
#sns.countplot("Embarked",data=df2)

In [4]:
print(df2.Cabin.mode())
print(df2.Fare.mode())

Age_mdeian=df2.Age.median()
df2.Age=df2.Age.fillna(Age_mdeian)

0    C23 C25 C27
dtype: object
0    8.05
dtype: float64


In [5]:
# 欠損値の補完
df2.Fare=df2.Fare.fillna(8.05)
df2.Cabin=df2.Cabin.fillna("C23")
df2.Embarked=df2.Embarked.fillna("S")

In [6]:
df2.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [7]:
#使わないカラムを削除する
df3=df2.drop(columns=["Name","Ticket","Fare"])
df3.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch
count,1309.000000,891.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,0.383838,2.294882,29.503186,0.498854,0.385027
std,378.020061,0.486592,0.837836,12.905241,1.041658,0.865560
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,22.000000,0.000000,0.000000
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000
75%,982.000000,1.000000,3.000000,35.000000,1.000000,0.000000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000


In [8]:
#カテゴリカル変数の数値変換
tmp_Sex=pd.get_dummies(df3["Sex"],prefix="Sex")
df3=pd.concat([df3,tmp_Sex],axis=1).drop(columns="Sex")

tmp_Cabin=pd.get_dummies(df3["Cabin"],prefix="Cabin")
df3=pd.concat([df3,tmp_Cabin],axis=1).drop(columns="Cabin")

tmp_Embarked=pd.get_dummies(df3["Embarked"],prefix="Embarked")
df3=pd.concat([df3,tmp_Embarked],axis=1).drop(columns="Embarked")
df3.sample(10)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A11,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
714,715,0.0,2,52.00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1081,1082,NaN,2,34.00,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
100,101,0.0,3,28.00,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
644,645,1.0,3,0.75,2,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
331,332,0.0,1,45.50,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
311,312,1.0,1,18.00,2,2,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
174,175,0.0,1,56.00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
73,74,0.0,3,26.00,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
979,980,NaN,3,28.00,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
104,105,0.0,3,37.00,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
# 学習データに分割した結果を変数trainに格納する
train=df3[~df3.Survived.isnull()]

# テストデータに分割した結果を変数trainに格納する
test=df3[df3.Survived.isnull()]

#trainのデータ
train_x=train.drop(columns=["Survived","PassengerId"])
train_y=train["Survived"]

In [10]:
clf=LGBMClassifier(boosting_type="gbdt",
                      objective="binary",
                      metric="auc",
                      seed=0,
                      n_estimators=480,
                      num_leaves=5,
                      max_depth=3,
                      learning_rate=0.01)

In [11]:
clf.fit(train_x,train_y)

X_test=test.drop(columns=["Survived","PassengerId"])
y_predict=clf.predict(X_test)

In [12]:
result=pd.DataFrame(y_predict)

#列名に名前をつける
result2=result.rename(columns={result.columns[0]: "Survived"})

#PassngerIdを合わせる
l=[]
l2=[]
t_id=test_df.loc[:,"PassengerId"]
for i in range(len(t_id)):
               tmp={
                   "PassengerId":t_id[i]
               }
               l.append(tmp);
               
l2=pd.DataFrame(l);
result3=pd.concat([l2,result2],axis=1)


#予測結果を整数に変換する
result3.Survived=result3.Survived.astype(int)
result3.PassengerId=result3.PassengerId.astype(int)

In [13]:
# CSVの作成
result3.to_csv("predict6.csv",index=False)